In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples
from src.predictionModule.MachineModels import MachineModels 

import numpy as np
import datetime
import pandas as pd
import polars as pl
import optuna

import logging
formatted_date = datetime.datetime.now().strftime("%d%b%y_%H%M").lower()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]

#Output File handler
formatted_str = f"notebook-lstm-optuna-{formatted_date}"
file_handler = logging.FileHandler(f"{formatted_str}.log", mode="w")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Usage
logger.info("This will print to the notebook's output cell")

2025-08-16 17:17:58,212 - This will print to the notebook's output cell


c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Default params (keeps previous behavior where params corresponds to idxAfterPrediction=5)
params_default = {
    "idxAfterPrediction": 5,
    'timesteps': 90,
    'target_option': 'last',
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 10,
    
    "LSTM_units": 32,
    "LSTM_num_layers": 3,
    "LSTM_dropout": 0.001,
    "LSTM_recurrent_dropout": 0.001,
    "LSTM_learning_rate": 0.001,
    "LSTM_optimizer": "adam",
    "LSTM_bidirectional": True,
    "LSTM_batch_size": 2**12,
    "LSTM_epochs": 10,
    "LSTM_l1": 0.001,
    "LSTM_l2": 0.001,
    "LSTM_inter_dropout": 0.001,
    "LSTM_input_gaussian_noise": 0.001,
    "LSTM_conv1d": True,
    "LSTM_conv1d_kernel_size": 3,
    "LSTM_loss": "mse",
}

In [3]:
stock_group = "group_regOHLCV_over5years"
eval_date = datetime.date(year=2025, month=6, day=13)
split_date = datetime.date(year=2023, month=12, day=31)

studytime = 60*60*1
studyname = f"sandbox_lstm_optuna_{formatted_str}"

In [12]:
def objective(trial: optuna.Trial) -> float:
    opt_params = params_default.copy()
    opt_params["year_start"] = trial.suggest_int("year_start", 2019, 2023)
    opt_params["idxAfterPrediction"] = trial.suggest_int("idxAfterPrediction", 3, 5, step=1)
    opt_params["LoadupSamples_time_inc_factor"] = trial.suggest_int("LoadupSamples_time_inc_factor", 21, 71, step=5)
    opt_params["timesteps"] = trial.suggest_int("timesteps", 50, 90, step=5)
    opt_params["LSTM_units"] = 16
    opt_params["LSTM_num_layers"] = 1
    opt_params["LSTM_learning_rate"] = trial.suggest_float("LSTM_learning_rate", 1e-5, 1e-3, log=True)
    opt_params["LSTM_epochs"] = 2
    opt_params["LSTM_l1"] = trial.suggest_float("LSTM_l1", 1e-5, 1e-3, log=True)
    opt_params["LSTM_l2"] = trial.suggest_float("LSTM_l2", 1e-4, 1e-1, log=True)
    opt_params["LSTM_dropout"] = trial.suggest_float("LSTM_dropout", 1e-5, 1e-3, log=True)
    opt_params["LSTM_inter_dropout"] = trial.suggest_float("LSTM_inter_dropout", 1e-4, 1e-1, log=True)
    opt_params["LSTM_recurrent_dropout"] = trial.suggest_float("LSTM_recurrent_dropout", 1e-4, 1e-1, log=True)
    opt_params["LSTM_conv1d_kernel_size"] = 3
    opt_params["is_single_feature"] = trial.suggest_categorical("is_single_feature", [True, False])

    ls = LoadupSamples(
        train_start_date=datetime.date(year=opt_params["year_start"], month=1, day=1),
        test_dates=[eval_date],
        group=stock_group,
        group_type="Time",
        params=opt_params,
    )
    ls.load_samples(main_path = "../src/featureAlchemy/bin/")

    ls.split_dataset(
        start_date=datetime.date(year=opt_params["year_start"], month=1, day=1),
        last_train_date=split_date,
        last_test_date=eval_date,
    )

    Xtrain = ls.train_Xtime
    ytrain = ls.train_ytime
    Xtest = ls.test_Xtime
    ytest = ls.test_ytime

    true_res = ls.meta_pl_test

    Xtrain = Xtrain[:, -opt_params["timesteps"]:, :]
    Xtest = Xtest[:, -opt_params["timesteps"]:, :]

    if opt_params["is_single_feature"]:
        Xtrain = Xtrain[:, :, [0]]
        Xtest = Xtest[:, :, [0]]

    starttime = datetime.datetime.now()
    mm = MachineModels(opt_params)
    model_lstm, res_dict = mm.run_LSTM_torch(Xtrain, ytrain, Xtest, ytest, device="cuda")
    preds = mm.predict_LSTM_torch(model_lstm, Xtest, batch_size=opt_params["LSTM_batch_size"], device="cuda")
    endtime = datetime.datetime.now()
    
    q = 0.98
    mask_pred_above = (preds >= np.quantile(preds, q))
    mask_pred_below = (preds <= np.quantile(preds, 1-q))

    true_res_masked_above = true_res.filter(pl.Series(mask_pred_above))
    true_res_masked_below = true_res.filter(pl.Series(mask_pred_below))

    score = (np.mean(true_res_masked_above['target_ratio'].to_numpy())) ** (1/opt_params["idxAfterPrediction"])

    # Log some results
    def quant_dis_in_mask(mask: np.ndarray, q: float) -> int:
        if not mask.any():
            return len(mask)
        return np.quantile(np.abs(np.diff(np.where(mask)[0])), q)

    logger.info(f"Trial {trial.number} with params: {opt_params}")
    logger.info(f"  Duration: {endtime - starttime}")
    logger.info(f"  Val RSME adjusted: {res_dict['val_rmse']/opt_params['LoadupSamples_time_inc_factor']:.4f}")
    logger.info(f"  Mean all prediction: {np.mean(true_res['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Mean above prediction: {np.mean(true_res_masked_above['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Mean below prediction: {np.mean(true_res_masked_below['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Quantile 0.99 in mask above: {quant_dis_in_mask(mask_pred_above, 0.99)}")
    logger.info(f"  Quantile 0.99 in mask below: {quant_dis_in_mask(mask_pred_below, 0.99)}")
    logger.info(f"  Ratio for quantile to length above: {quant_dis_in_mask(mask_pred_above, 0.99) / len(mask_pred_above):.4f}")
    logger.info(f"  Ratio for quantile to length below: {quant_dis_in_mask(mask_pred_below, 0.99) / len(mask_pred_below):.4f}")
    logger.info(f"  Score: {score:.4f}")

    return score

optuna.logging.enable_propagation()
sampler = optuna.samplers.TPESampler(n_startup_trials=15)
study = optuna.create_study(
    study_name = studyname,
    storage="sqlite:///sandbox_optuna.db",
    direction="maximize",
    load_if_exists=True,
    sampler=sampler,
)
study.optimize(objective, timeout=studytime)

logger.info(f"Best parameters: {study.best_params}")
logger.info(f"Best score: {study.best_value}")

df: pd.DataFrame = study.trials_dataframe()
logger.info("\nTrials DataFrame:")
logger.info(df.sort_values("value").to_string())

param_importances = optuna.importance.get_param_importances(study)
logger.info("Parameter Importances:")
for key, value in param_importances.items():
    logger.info(f"{key}: {value}")

[I 2025-08-16 19:24:00,757] Using an existing study with name 'sandbox_lstm_optuna_notebook-lstm-optuna-16aug25_1717' instead of creating a new one.


2025-08-16 19:24:00,757 - Using an existing study with name 'sandbox_lstm_optuna_notebook-lstm-optuna-16aug25_1717' instead of creating a new one.
2025-08-16 19:24:11,796 - NaN values found in training time features. 28 Samples removed.


Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

2025-08-16 19:24:58,083 - Epoch 1/2 — Train RMSE: 0.4179 — Validation RMSE: 0.3311


Epochs:  50%|█████     | 1/2 [00:31<00:31, 31.59s/it]

...

2025-08-16 20:25:19,783 - Epoch 2/2 — Train RMSE: 0.3246 — Validation RMSE: 0.3112


Epochs: 100%|██████████| 2/2 [01:03<00:00, 31.65s/it]


2025-08-16 20:25:23,434 - Trial 92 with params: {'idxAfterPrediction': 4, 'timesteps': 70, 'target_option': 'last', 'LoadupSamples_time_scaling_stretch': True, 'LoadupSamples_time_inc_factor': 26, 'LSTM_units': 16, 'LSTM_num_layers': 1, 'LSTM_dropout': 0.0007299806156663939, 'LSTM_recurrent_dropout': 0.0031753923949816554, 'LSTM_learning_rate': 0.0007029953343934389, 'LSTM_optimizer': 'adam', 'LSTM_bidirectional': True, 'LSTM_batch_size': 4096, 'LSTM_epochs': 2, 'LSTM_l1': 1.1069358688662353e-05, 'LSTM_l2': 0.0001194814732092283, 'LSTM_inter_dropout': 0.0006040021491262095, 'LSTM_input_gaussian_noise': 0.001, 'LSTM_conv1d': True, 'LSTM_conv1d_kernel_size': 3, 'LSTM_loss': 'mse', 'year_start': 2019, 'is_single_feature': False}
2025-08-16 20:25:23,434 -   Duration: 0:01:07.178940
2025-08-16 20:25:23,440 -   Val RSME adjusted: 0.0120
2025-08-16 20:25:23,441 -   Mean all prediction: 1.0010
2025-08-16 20:25:23,441 -   Mean above prediction: 1.0074
2025-08-16 20:25:23,441 -   Mean below pred

[I 2025-08-16 20:25:23,530] Trial 92 finished with value: 1.001845047291454 and parameters: {'year_start': 2019, 'idxAfterPrediction': 4, 'LoadupSamples_time_inc_factor': 26, 'timesteps': 70, 'LSTM_learning_rate': 0.0007029953343934389, 'LSTM_l1': 1.1069358688662353e-05, 'LSTM_l2': 0.0001194814732092283, 'LSTM_dropout': 0.0007299806156663939, 'LSTM_inter_dropout': 0.0006040021491262095, 'LSTM_recurrent_dropout': 0.0031753923949816554, 'is_single_feature': False}. Best is trial 10 with value: 1.0025323942546822.


2025-08-16 20:25:23,530 - Trial 92 finished with value: 1.001845047291454 and parameters: {'year_start': 2019, 'idxAfterPrediction': 4, 'LoadupSamples_time_inc_factor': 26, 'timesteps': 70, 'LSTM_learning_rate': 0.0007029953343934389, 'LSTM_l1': 1.1069358688662353e-05, 'LSTM_l2': 0.0001194814732092283, 'LSTM_dropout': 0.0007299806156663939, 'LSTM_inter_dropout': 0.0006040021491262095, 'LSTM_recurrent_dropout': 0.0031753923949816554, 'is_single_feature': False}. Best is trial 10 with value: 1.0025323942546822.
2025-08-16 20:25:23,534 - Best parameters: {'year_start': 2023, 'idxAfterPrediction': 4, 'LoadupSamples_time_inc_factor': 46, 'timesteps': 50, 'LSTM_learning_rate': 8.127345854411313e-05, 'LSTM_l1': 0.00048621558900400493, 'LSTM_l2': 0.00040047764362209144, 'LSTM_dropout': 6.919307928559869e-05, 'LSTM_inter_dropout': 0.00011939531126332565, 'LSTM_recurrent_dropout': 0.004360369703847029, 'is_single_feature': False}
2025-08-16 20:25:23,537 - Best score: 1.0025323942546822
2025-08-1

In [5]:
df.to_parquet(f"sandbox_lstm_optuna_{formatted_str}.parquet", index=False)

In [6]:
# Run with best parameters
best_params = {**params_default, **study.best_params.copy()}
best_params["LSTM_units"] = 16
best_params["LSTM_epochs"] = 2
best_params["LSTM_conv1d_kernel_size"] = 3
best_params["LSTM_num_layers"] = 1

ls = LoadupSamples(
    train_start_date=datetime.date(year=best_params["year_start"], month=1, day=1),
    test_dates=[eval_date],
    group=stock_group,
    group_type="Time",
    params=best_params,
)
ls.load_samples(main_path="../src/featureAlchemy/bin/")

ls.split_dataset(
    start_date=datetime.date(year=best_params["year_start"], month=1, day=1),
    last_train_date=split_date,
    last_test_date=eval_date,
)

Xtrain = ls.train_Xtime
ytrain = ls.train_ytime
Xtest = ls.test_Xtime
ytest = ls.test_ytime

Xtrain = Xtrain[:, -best_params["timesteps"]:, :]
Xtest = Xtest[:, -best_params["timesteps"]:, :]

if best_params["is_single_feature"]:
    Xtrain = Xtrain[:, :, [0]]
    Xtest = Xtest[:, :, [0]]

mm = MachineModels(best_params)
model_lstm, res_dict = mm.run_LSTM_torch(Xtrain, ytrain, Xtest, ytest, device="cuda")
preds = mm.predict_LSTM_torch(model_lstm, Xtest, batch_size=best_params["LSTM_batch_size"], device="cuda")



2025-08-16 18:18:25,292 - NaN values found in training time features. 28 Samples removed.


Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

2025-08-16 18:18:42,097 - Epoch 1/2 — Train RMSE: 0.7566 — Validation RMSE: 0.6959


Epochs:  50%|█████     | 1/2 [00:11<00:11, 11.01s/it]

2025-08-16 18:18:53,391 - Epoch 2/2 — Train RMSE: 0.6707 — Validation RMSE: 0.6077


Epochs: 100%|██████████| 2/2 [00:22<00:00, 11.15s/it]


In [11]:
q = 0.98
mask_pred_above = (preds >= np.quantile(preds, q))
mask_pred_below = (preds <= np.quantile(preds, 1-q))
true_res = ls.meta_pl_test
true_res_masked_above = true_res.filter(pl.Series(mask_pred_above))
true_res_masked_below = true_res.filter(pl.Series(mask_pred_below))
score = (np.mean(true_res_masked_above['target_ratio'].to_numpy())) ** (1/best_params["idxAfterPrediction"])
# Log some results
def quant_dis_in_mask(mask: np.ndarray, q: float) -> int:
    if not mask.any():
        return len(mask)
    return np.quantile(np.abs(np.diff(np.where(mask)[0])), q)
logger.info(f"  Val RSME adjusted: {res_dict['val_rmse']/best_params['LoadupSamples_time_inc_factor']:.4f}")
logger.info(f"  Mean all prediction: {np.mean(true_res['target_ratio'].to_numpy()):.4f}")
logger.info(f"  Mean above prediction: {np.mean(true_res_masked_above['target_ratio'].to_numpy()):.4f}")
logger.info(f"  Mean below prediction: {np.mean(true_res_masked_below['target_ratio'].to_numpy()):.4f}")
logger.info(f"  Quantile 0.99 in mask above: {quant_dis_in_mask(mask_pred_above, 0.99)}")
logger.info(f"  Quantile 0.99 in mask below: {quant_dis_in_mask(mask_pred_below, 0.99)}")
logger.info(f"  Ratio for quantile to length above: {quant_dis_in_mask(mask_pred_above, 0.99) / len(mask_pred_above):.4f}")
logger.info(f"  Ratio for quantile to length below: {quant_dis_in_mask(mask_pred_below, 0.99) / len(mask_pred_below):.4f}")
logger.info(f"  Score: {score:.4f}")

2025-08-16 18:33:38,602 -   Val RSME adjusted: 0.0132
2025-08-16 18:33:38,604 -   Mean all prediction: 1.0010
2025-08-16 18:33:38,605 -   Mean above prediction: 1.0101
2025-08-16 18:33:38,605 -   Mean below prediction: 1.0001
2025-08-16 18:33:38,608 -   Quantile 0.99 in mask above: 450.27999999999884
2025-08-16 18:33:38,609 -   Quantile 0.99 in mask below: 427.0
2025-08-16 18:33:38,611 -   Ratio for quantile to length above: 0.0007
2025-08-16 18:33:38,612 -   Ratio for quantile to length below: 0.0006
2025-08-16 18:33:38,612 -   Score: 1.0025
